In [8]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
import os
from re import search

In [49]:
def get_subdirectories(folder_path):
    subdirectories = []
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if os.path.isdir(item_path):
            subdirectories.append(item)
    return subdirectories

def get_files_in_subdirectories(folder_path, file_extension='', file_contains=''):
    files = []
    for root, directories, filenames in os.walk(folder_path):
        for filename in filenames:
            if file_extension == '' and file_contains == '':
                files.append(os.path.join(root, filename))
            elif file_extension != '' and file_contains == '':
                if filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
            elif file_extension == '' and file_contains != '':
                if file_contains in filename:
                    files.append(os.path.join(root, filename))
            else:
                if file_contains in filename and filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
    return files

def get_num(text):
    if isinstance(text, list):
        numbers = []
        for item in text:
            number_str = search(r'\d+(?=[^\d]*\.csv)', str(item))[0]
            if number_str:
                numbers.append(number_str)
        return numbers
    else:
        number_str = search(r'\d+(?=[^\d]*\.csv)', str(text))[0]
        return number_str if number_str else None

# def read_csv_to_tensor(file_path, tensor_name):
#     # Read CSV file as a pandas DataFrame
#     dataframe = pd.read_csv(file_path)
#     # Convert DataFrame to a tensor
#     tensor = torch.tensor(dataframe.values)
#     # Assign the tensor to the specified variable name
#     locals()[tensor_name] = tensor

def read_csv_to_tensor(file_path, tensor_name, tensors_dict):
    # Read CSV file as a pandas DataFrame
    dataframe = pd.read_csv(file_path)

    # Convert DataFrame to a tensor
    tensor = torch.tensor(dataframe.values)

    # Store the tensor in the dictionary with the specified variable name
    tensors_dict[tensor_name] = tensor


folder_path = os.getcwd()+'/data/'
print(folder_path)
subdirectories = get_subdirectories(folder_path)
print(subdirectories)

file_extension = ['.csv','.csv.stepMixed']
label = ['data','labels']
tensors = {}

for i in subdirectories:
    folder_i = folder_path+i
    files_t = get_files_in_subdirectories(folder_i)
    files_n = list(dict.fromkeys(get_num(files_t)))
    for j in files_n:
        for index_k,k in enumerate(file_extension):
            files = get_files_in_subdirectories(folder_i,file_extension=k,file_contains=j)
            name_variable = i+'_'+j+'_'+label[index_k]
            print(f'variable: {name_variable} person: {i} file_extension: {k} file_path: {files}')
            tensor = read_csv_to_tensor(files[0],name_variable,tensors)




/Users/rodolfocacacho/Documents/Documents/MAI/1.Semester/Artificial Neural Networks and Cognitive Models/Kaggle competition dataset/ANN_Project/data/
['person_8', 'person_7', 'person_12', 'person_5', 'person_2', 'person_3', 'person_4', 'person_11', 'person_10']
variable: person_8_20230217130617_data person: person_8 file_extension: .csv file_path: ['/Users/rodolfocacacho/Documents/Documents/MAI/1.Semester/Artificial Neural Networks and Cognitive Models/Kaggle competition dataset/ANN_Project/data/person_8/ClippedawindaRecording_20230217130617.csv']
variable: person_8_20230217130617_labels person: person_8 file_extension: .csv.stepMixed file_path: ['/Users/rodolfocacacho/Documents/Documents/MAI/1.Semester/Artificial Neural Networks and Cognitive Models/Kaggle competition dataset/ANN_Project/data/person_8/awindaRecording_20230217130617.csv.stepMixed']
variable: person_7_20230206175311_data person: person_7 file_extension: .csv file_path: ['/Users/rodolfocacacho/Documents/Documents/MAI/1.S

In [46]:
file = '/Users/rodolfocacacho/Documents/Documents/MAI/1.Semester/Artificial Neural Networks and Cognitive Models/Kaggle competition dataset/ANN_Project/data/person_8/ClippedawindaRecording_20230217130617.csv'
test = 'tensorx'
tensor = read_csv_to_tensor(file,test)

# Access the tensor using the variable name
my_tensor = tensor[test]

# Print the tensor
print(my_tensor)



tensor([[ 0.0000e+00,  1.3706e+00,  3.0777e+00,  ...,  2.6021e-02,
         -2.5069e-02,  2.6772e-02],
        [ 0.0000e+00,  1.3807e+00,  3.0394e+00,  ...,  3.8649e-02,
         -3.8450e-02,  3.5676e-02],
        [ 0.0000e+00,  1.3783e+00,  2.9815e+00,  ...,  4.3459e-02,
         -3.8100e-02,  3.1424e-02],
        ...,
        [ 0.0000e+00,  6.1260e-01,  2.6757e+00,  ...,  4.0939e-02,
         -1.6002e-02, -6.7030e-03],
        [ 0.0000e+00,  6.0394e-01,  2.6538e+00,  ...,  1.6346e-02,
         -2.5056e-02, -1.3025e-02],
        [ 0.0000e+00,  6.0076e-01,  2.6006e+00,  ..., -1.7180e-03,
         -1.8781e-02, -2.0006e-02]], dtype=torch.float64)


In [50]:
tensors

{'person_8_20230217130617_data': tensor([[ 0.0000e+00,  1.3706e+00,  3.0777e+00,  ...,  2.6021e-02,
          -2.5069e-02,  2.6772e-02],
         [ 0.0000e+00,  1.3807e+00,  3.0394e+00,  ...,  3.8649e-02,
          -3.8450e-02,  3.5676e-02],
         [ 0.0000e+00,  1.3783e+00,  2.9815e+00,  ...,  4.3459e-02,
          -3.8100e-02,  3.1424e-02],
         ...,
         [ 0.0000e+00,  6.1260e-01,  2.6757e+00,  ...,  4.0939e-02,
          -1.6002e-02, -6.7030e-03],
         [ 0.0000e+00,  6.0394e-01,  2.6538e+00,  ...,  1.6346e-02,
          -2.5056e-02, -1.3025e-02],
         [ 0.0000e+00,  6.0076e-01,  2.6006e+00,  ..., -1.7180e-03,
          -1.8781e-02, -2.0006e-02]], dtype=torch.float64),
 'person_8_20230217130617_labels': tensor([[ 1176,  1226],
         [ 1273,  1319],
         [ 1363,  1411],
         ...,
         [29017, 29059],
         [29109, 29152],
         [29198, 29247]]),
 'person_7_20230206175311_data': tensor([[ 0.0000, -1.1777,  2.4653,  ...,  0.0664, -0.0304, -0.0522]